In [1]:
!pip install -q keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

dataset = pd.read_csv('/content/drive/MyDrive/Data/train.csv')
dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [4]:
dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
dataset = dataset.dropna()

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [7]:
dataset = dataset.reset_index()

In [8]:
dataset

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
X = dataset.drop('label',axis = 1)
Y = dataset['label']

In [12]:
## One Hot encoding

In [10]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,Dense
from keras.models import Sequential

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
messages = X.copy()
corpus = []
ps = PorterStemmer()

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [13]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [14]:
voc_size = 5000
onehot_rep = [one_hot(words,voc_size) for words in corpus]

In [15]:
onehot_rep

[[4646, 1850, 2137, 1827, 4376, 1622, 4622, 14, 2746, 4778],
 [516, 974, 2129, 1796, 3800, 2513, 2793],
 [621, 3516, 3339, 2597],
 [1524, 4128, 2588, 4578, 2207, 209],
 [4907, 3800, 3893, 400, 305, 1056, 3800, 917, 2350, 602],
 [3091,
  3356,
  3864,
  2074,
  717,
  4262,
  549,
  2952,
  2845,
  3931,
  4391,
  3062,
  51,
  2079,
  2793],
 [3960, 1078, 2031, 1618, 674, 2628, 452, 2081, 2908, 3301, 647],
 [4468, 1039, 1421, 2459, 1842, 405, 4262, 2071, 2908, 3301, 647],
 [122, 1500, 4115, 1749, 4097, 718, 4366, 4799, 4262, 4881],
 [1871, 1736, 3746, 404, 1211, 1169, 2411, 960],
 [4060, 2850, 1890, 2213, 4884, 314, 711, 23, 3698, 3508, 3269],
 [4578, 1347, 4376, 718, 4262, 1842],
 [4962, 3413, 3796, 1463, 1443, 2371, 2476, 1662, 3150],
 [3448, 1808, 934, 118, 2862, 1064, 165, 2908, 3301, 647],
 [3510, 419, 2606, 2138, 1146, 2908, 3301, 647],
 [3867, 1972, 2496, 4786, 2136, 3747, 2448, 605, 422, 2274],
 [4862, 220, 974],
 [4830, 2718, 2858, 1177, 4262, 191, 4272, 2793],
 [584, 2412, 21

In [16]:
sent_len = 20
emb_doc = pad_sequences(onehot_rep,padding = 'post',maxlen = sent_len)
emb_doc

array([[4646, 1850, 2137, ...,    0,    0,    0],
       [ 516,  974, 2129, ...,    0,    0,    0],
       [ 621, 3516, 3339, ...,    0,    0,    0],
       ...,
       [3280, 4066, 3291, ...,    0,    0,    0],
       [4905, 1842, 2339, ...,    0,    0,    0],
       [1228, 3600, 4579, ...,    0,    0,    0]], dtype=int32)

In [20]:
## Model buliding Using LSTM RNN

In [17]:
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(voc_size,40,input_length = sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
len(emb_doc),Y.shape

(18285, (18285,))

In [19]:
import numpy as np

X_final=np.array(emb_doc)
y_final=np.array(Y)

In [ ]:
X_final

array([[4862,  390, 3775, ...,    0,    0,    0],
       [4893, 2979, 2777, ...,    0,    0,    0],
       [ 428, 4102,   37, ...,    0,    0,    0],
       ...,
       [1238,  104, 4961, ...,    0,    0,    0],
       [3079, 1626, 4251, ...,    0,    0,    0],
       [1420,  103,   38, ...,    0,    0,    0]], dtype=int32)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)

In [22]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 15ms/step - loss: 0.3235 - accuracy: 0.8402 - val_loss: 0.2026 - val_accuracy: 0.9210
Epoch 2/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1543 - accuracy: 0.9417 - val_loss: 0.1956 - val_accuracy: 0.9246
Epoch 3/10
192/192 [==============================] - 2s 12ms/step - loss: 0.1135 - accuracy: 0.9609 - val_loss: 0.2041 - val_accuracy: 0.9223
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0869 - accuracy: 0.9701 - val_loss: 0.1991 - val_accuracy: 0.9221
Epoch 5/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0667 - accuracy: 0.9788 - val_loss: 0.2290 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0570 - accuracy: 0.9812 - val_loss: 0.2760 - val_accuracy: 0.9191
Epoch 7/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0468 - accuracy: 0.9855 - val_loss: 0.3150 - val_accuracy: 0.9085

In [23]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")


In [25]:
## Model Validation

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score

print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[3103  286]
 [ 219 2427]]
0.9163214581607291
